In [1]:
import json
import pandas as pd

# Extract the data from Airbnb json dataset

d=open('sample_airbnb.json','r')
airbnb_data=json.load(d)

In [24]:
# Extract Hotels info and transformed into dataframe

def Hotels_info():
        
        hotel_data=[]

        for i in airbnb_data:
                data=dict(
                        id=i['_id'],
                        listing_url=i['listing_url'],
                        name=i['name'],
                        description=i['description'],
                        neighborhood_overview=i['neighborhood_overview'],
                        transit=i['transit'],
                        street=i['address']['street'],
                        suburb=i['address']['suburb'],
                        government_area=i['address']['government_area'],
                        market=i['address']['market'],
                        country=i['address']['country'],
                        country_code=i['address']['country_code'],
                        coordinates=i['address']['location'].get('coordinates'),
                        price=i['price'],
                        weekly_price=i.get('weekly_price'),
                        monthly_price=i.get('monthly_price'),
                        security_deposit=i.get('security_deposit'),
                        cleaning_fee=i.get('cleaning_fee'),
                        cancellation_policy=i['cancellation_policy'],
                        )
                hotel_data.append(data)

        Hotel_info=pd.DataFrame(hotel_data)
        Hotel_info[['security_deposit','cleaning_fee','weekly_price','monthly_price']]=Hotel_info[['security_deposit','cleaning_fee','weekly_price','monthly_price']].fillna(0)
        Hotel_info[['price','security_deposit','cleaning_fee','weekly_price','monthly_price']]=Hotel_info[['price','security_deposit','cleaning_fee','weekly_price','monthly_price']].astype(dtype='int64')
        Hotel_info['coordinates'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Hotel_info['coordinates']]

        
        return Hotel_info

In [4]:
# Extract Rooms info and transformed into dataframe

def Room_info():
    
    room_data=[]

    for i in airbnb_data:
        data=dict(
            id=i['_id'],
            images=i['images']['picture_url'],
            property_type=i['property_type'],
            room_type=i['room_type'],
            bed_type=i['bed_type'],
            minimum_nights=i['minimum_nights'],
            maximum_nights=i['maximum_nights'],
            accommodates=i['accommodates'],
            bedrooms=i.get('bedrooms'),
            beds=i.get('beds'),
            bathrooms=i.get('bathrooms'),
            amenities=i['amenities'],
            guests_included=i['guests_included'],
            availability_30=i['availability']['availability_30'],
            availability_60=i['availability']['availability_60'],
            availability_90=i['availability']['availability_90'],
            availability_365=i['availability']['availability_365']
            )
        room_data.append(data)

    Room_info=pd.DataFrame(room_data)
    Room_info[['bedrooms','beds','bathrooms']]=Room_info[['bedrooms','beds','bathrooms']].fillna(0)
    Room_info[['bedrooms','beds','bathrooms','guests_included']]=Room_info[['bedrooms','beds','bathrooms','guests_included']].astype(dtype='int64')
    Room_info['amenities'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Room_info['amenities']]

    return Room_info


In [6]:
# Extract Hosts info and transformed into dataframe

def Host_info():

    host_data=[]

    for i in airbnb_data:
        data=dict(
            id=i['_id'],
            host_id=i['host']['host_id'],
            host_url=i['host']['host_url'],
            host_name=i['host']['host_name'],
            host_location=i['host']['host_location'],
            host_about=i['host']['host_about'],
            host_picture_url=i['host']['host_picture_url'],
            host_neighbourhood=i['host']['host_neighbourhood'],
            host_response_rate=i['host'].get('host_response_rate'),
            host_total_listings=i['host']['host_total_listings_count'],
            host_verifications=i['host']['host_verifications']
            )
        host_data.append(data)

    Host_info=pd.DataFrame(host_data)
    Host_info['host_response_rate']=Host_info['host_response_rate'].fillna(0).astype(dtype='int64')
    Host_info['host_verifications'] = [', '.join(map(str, x)) if isinstance(x, list) else '' for x in Host_info['host_verifications']]
    

    return Host_info

In [8]:
# Extract Reviews info and transformed into dataframe

def Review_info():

    review_data=[]

    for i in airbnb_data:
        data=dict(
            id=i['_id'],
            first_review=i.get('first_review'),
            last_review=i.get('last_review'),
            number_of_reviews=i['number_of_reviews'],
            overall_score=i['review_scores'].get('review_scores_accuracy'),
            cleanliness_score=i['review_scores'].get('review_scores_cleanliness'),
            checkin_score=i['review_scores'].get('review_scores_checkin'),
            communication_score=i['review_scores'].get('review_scores_communication'),
            location_score=i['review_scores'].get('review_scores_communication'),
            value_score=i['review_scores'].get('review_scores_value'),
            rating=i['review_scores'].get('review_scores_rating')
        )
        review_data.append(data)

    Review_info=pd.DataFrame(review_data)
    Review_info[['first_review','last_review']]=Review_info[['first_review','last_review']].fillna('Not Available')
    Review_info[['overall_score','cleanliness_score','checkin_score','communication_score','location_score','value_score','rating']]=Review_info[['overall_score','cleanliness_score','checkin_score','communication_score','location_score','value_score','rating']].fillna(0).astype(dtype='int64')

    return Review_info

In [17]:
# Extract Review commands info and transformed into dataframe

def Review_comments_info():

    comments_data=[]

    for j in airbnb_data:
        r_list=j.get('reviews',[])
        for i in r_list:
            data=dict(
                id=i.get('listing_id'),
                comment_id=i.get('_id'),
                date=i.get('date'),
                reviewer_id=i.get('reviewer_id'),
                reviewer_name=i.get('reviewer_name'),
                comments=i.get('comments')
                )
            comments_data.append(data)

    Review_comments_info=pd.DataFrame(comments_data)
    Review_comments_info[['reviewer_name','comments']]=Review_comments_info[['reviewer_name','comments']].fillna('Not Available')

    return Review_comments_info

In [22]:
#mysql connetor to connect with python
import mysql.connector

mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True,)

#create database and use for table creation
mycursor.execute('create database if not exists airbnb')
mycursor.execute('use airbnb')

In [23]:
# Created Hotels_info table in sql database and inserted the values 

mycursor.execute('''create table if not exists hotels_info (id VARCHAR(50) PRIMARY KEY, listing_url VARCHAR(500),
                name VARCHAR(100), description TEXT, neighborhood_overview TEXT, transit TEXT, street VARCHAR(100),
                suburb VARCHAR(100), government_area VARCHAR(100), market VARCHAR(100), country VARCHAR(100),
                country_code VARCHAR(50), coordinates VARCHAR(100), price INT, weekly_price INT, monthly_price INT, 
                security_deposit INT, cleaning_fee INT, cancellation_policy VARCHAR(50))
                    ''')

insert_value='''INSERT INTO hotels_info (id, listing_url, name, description, neighborhood_overview, transit, street,suburb,
                government_area, market, country, country_code, coordinates, price, weekly_price, monthly_price,
                security_deposit, cleaning_fee, cancellation_policy) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

data = [tuple(row) for row in Hotels_info().values]

mycursor.executemany(insert_value,data)
mydb.commit()

In [27]:
# Created Rooms_info table in sql database and inserted the values 

mycursor.execute('''create table if not exists rooms_info (id VARCHAR(50), images VARCHAR(500), property_type VARCHAR(50),
                room_type VARCHAR(50), bed_type VARCHAR(50), minimum_nights VARCHAR(10), maximum_nights VARCHAR(10),
                accommodates INT, bedrooms INT, beds INT, bathrooms INT, amenities TEXT, guests_included INT,
                availability_30 INT, availability_60 INT, availability_90 INT, availability_365 INT,
                FOREIGN KEY(id) REFERENCES hotels_info(id))
                ''') 

insert_value='''INSERT INTO rooms_info (id, images, property_type, room_type, bed_type, minimum_nights, maximum_nights,
                accommodates, bedrooms, beds, bathrooms, amenities, guests_included, availability_30, availability_60,
                availability_90, availability_365)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

data= [tuple(row) for row in Room_info().values]

mycursor.executemany(insert_value,data)

mydb.commit()

In [29]:
# Created Hosts_info table in sql database and inserted the values 

mycursor.execute('''create table if not exists host_info (id VARCHAR(50), host_id VARCHAR(50), host_url VARCHAR(500),
                host_name VARCHAR(100), host_location VARCHAR(100), host_about TEXT, host_picture_url VARCHAR(500),
                host_neighbourhood VARCHAR(500), host_response_rate INT, host_total_listings INT, host_verifications VARCHAR(500),
                FOREIGN KEY(id) REFERENCES hotels_info(id))
                ''')

insert_value='''INSERT INTO host_info (id, host_id, host_url, host_name, host_location, host_about, host_picture_url,
                host_neighbourhood, host_response_rate, host_total_listings, host_verifications)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

data= [tuple(row) for row in Host_info().values]

mycursor.executemany(insert_value,data)

mydb.commit()

In [30]:
# Created Reviwes_info table in sql database and inserted the values 

mycursor.execute('''create table if not exists reviews_info (id VARCHAR(50), first_review DATETIME, last_review DATETIME,
                number_of_reviews INT, overall_score INT, cleanliness_score INT, checkin_score INT, communication_score INT,
                location_score INT, value_score INT,rating INT, FOREIGN KEY(id) REFERENCES hotels_info(id))
                ''')

insert_value='''INSERT INTO reviews_info (id, first_review, last_review, number_of_reviews, overall_score, cleanliness_score,
                checkin_score, communication_score, location_score, value_score, rating)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

data= [tuple(row) for row in Review_info().values]

mycursor.executemany(insert_value,data)

mydb.commit() 

In [40]:
# Created Comments_info table in sql database and inserted the values 

mycursor.execute('''create table if not exists comments_info (id VARCHAR(50), comment_id VARCHAR(50), date DATETIME,
                reviewer_id VARCHAR(50), reviewer_name VARCHAR(50), comments TEXT, FOREIGN KEY(id) REFERENCES hotels_info(id))
                ''')

insert_value='''INSERT INTO comments_info (id, comment_id, date, reviewer_id, reviewer_name ,comments)
                VALUES (%s,%s,%s,%s,%s,%s)'''

data= [tuple(row) for row in Review_comments_info().values]

mycursor.executemany(insert_value,data)

mydb.commit()